## *Парсинг* названий научных статей с одной из популярных онлайн библиотек.

#### Для примера произведен отбор **20 тыс.** заголовков, из которых сформирован DataFrame.

#### Полученный DataFrame предназначен для проверки работы модели **multiT5-3tasks-titles_scientific_articles.pth**


In [ ]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time

In [ ]:
main_url = 'https://cyberleninka.ru' # На мой взгляд, лучший российский онлайн ресурс в своем сегменте. Браво создателям!

#### Для начала напишем функцию для определения количества страниц

In [ ]:
def get_number_of_pages(main_url, base_link):
    new_url = main_url + base_link
    req = requests.get(new_url).text
    soup = BeautifulSoup(req, 'html.parser')

    link_last_page = soup.find('a', class_='icon').get('href').split('/')
    last_page = [int(el) for el in link_last_page if el.isdigit()]
    return last_page[-1]


#### Далее напишем основную функцию по парсингу названий научных статей

In [ ]:
def titles_scientific_articles(main_url, pages):
    titles_list = []  # Список для хранения названий статей
    session = requests.Session()  # Создаем сессию для повторного использования соединений

    req = session.get(main_url).text
    soup = BeautifulSoup(req, 'html.parser')

    targets = soup.find_all('div', class_=['half', 'half-right'])

    count = 0  # Счетчик добавленных названий статей
    start_time = time.time()

    for el in targets:
        direction = el.find_all('a')

        for article in direction:
            base_link = article.get('href')  # Получаем базовый href

            if base_link is None:  # Проверяем, что link не None
                continue

            if pages == 'all':
                number_of_pages = get_number_of_pages(main_url, base_link)
                time.sleep(0.25)
            else:
                number_of_pages = pages

            for page in range(1, number_of_pages + 1):  # Включаем последнюю страницу
                new_url = main_url + base_link + f'/{page}'  # Формируем новый URL

                try:
                    new_soup = BeautifulSoup(session.get(new_url).text, 'html.parser')
                    target_lst = new_soup.find_all('div', class_='title')
                    for name in target_lst:
                        titles_list.append(name.text)  # Добавляем название статьи в список
                        count += 1  # Увеличиваем счетчик

                        # Проверяем, достигли ли мы 20000 добавленных названий
                        if count >= 20000:
                            print(f"Добавлено {count} статей. Завершение функции.")
                            titles_df = pd.DataFrame(titles_list, columns=['name'])  # Создаем DataFrame один раз из списка
                            return titles_df.reset_index(drop=True)

                        # Каждые 1000 статей выводим информацию о процессе
                        if count % 1000 == 0:
                            print(f"{count} статей внесено в список.")
                            execution_time = time.time() - start_time
                            print(f"Время выполнения {count} строк: {execution_time:.2f} секунд")

                        time.sleep(0.25)  # Задержка между запросами
                except Exception as e:
                    print(f"Ошибка при обработке {new_url}: {e}")

    titles_df = pd.DataFrame(titles_list, columns=['name'])  # Создаем DataFrame один раз из списка
    return titles_df.reset_index(drop=True)

# Вызов функции
titles_df = titles_scientific_articles(main_url, pages='all')
print(titles_df)


In [ ]:
titles_df.head()

In [ ]:
titles_df.to_csv("titles_scientific_articles_Leninka.csv", index=False)